1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

# If haven't downloaded it yet, please get the data file with wget
!curl C:/Users/laura/ScientificComputingWithPython2023/data_000637.txt https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [1]:
import pandas as pd # standard naming convention
import numpy as np
import random as r
#print(os.path.exists("data_000637.txt"))
file_name ="data_000637.txt"
data = pd.read_csv(file_name)
print(data)
Nmax=len(data)
Nmin=10000
N=r.randint(Nmin,Nmax)
print(N)
data=data[0:N]
print(data)

         HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0           1     0          123  3869200167        2374        26
1           1     0          124  3869200167        2374        27
2           1     0           63  3869200167        2553        28
3           1     0           64  3869200167        2558        19
4           1     0           64  3869200167        2760        25
...       ...   ...          ...         ...         ...       ...
1310715     1     0           62  3869211171         762        14
1310716     1     1            4  3869211171         763        11
1310717     1     0           64  3869211171         764         0
1310718     1     0          139  3869211171         769         0
1310719     1     0           61  3869211171         762        18

[1310720 rows x 6 columns]
538001
        HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0          1     0          123  3869200167        2374        26
1          1     0          1

2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [2]:
#print(data)
BX_C=data.loc[:,['BX_COUNTER']]
#print(BX_COUNTER)
a=0
#print(BX_COUNTER.iat[1,0])

#for i in range(len(BX_COUNTER)-1):
#    #print(BX_COUNTER.iat[i,0])
#    if BX_COUNTER.iat[i+1,0]<BX_COUNTER.iat[i,0]:
#        #print(BX_COUNTER.iat[i+1,0],'p',BX_COUNTER.iat[i,0])
#        a+=1
#    i+=1
m=BX_C.max()
print('MAX',m[0])

MAX 3563


3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

*Hint:* introduce an offset to the absolute time such that the start of the data acquisition (i.e. the first entry) is zero.

In [3]:
import datetime
data['TIME_SERIES']= (data['TDC_CHANNEL']*25/30+data['BX_COUNTER']*25+25*m[0]*data['ORBIT_CNT'])
print("Minimo prima dell'offset:", data['TIME_SERIES'].min())
data['TIME_SERIES'] -= data['TIME_SERIES'].min()
print("Minimo dopo l'offset:", data['TIME_SERIES'].idxmin())
#print(data.iat[418706,6])

#data['TIME_SERIES'] = pd.to_datetime(data['TIME_SERIES'], unit='ns')
data


Minimo prima dell'offset: 344649004934977.5
Minimo dopo l'offset: 0


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIME_SERIES
0,1,0,123,3869200167,2374,26,0.000000e+00
1,1,0,124,3869200167,2374,27,8.125000e-01
2,1,0,63,3869200167,2553,28,4.425000e+03
3,1,0,64,3869200167,2558,19,4.550812e+03
4,1,0,64,3869200167,2760,25,9.600812e+03
...,...,...,...,...,...,...,...
537996,1,0,62,3869205614,749,5,4.851508e+08
537997,1,0,64,3869205614,756,21,4.851510e+08
537998,1,0,139,3869205614,760,0,4.851512e+08
537999,1,0,63,3869205614,758,17,4.851511e+08


4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

In [4]:
a=data['TIME_SERIES'].max()
print(a)
print('Duration',pd.to_timedelta(a))


485154597.5
Duration 0 days 00:00:00.485154597


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [5]:
#print(data.iloc[:,2])
f=data.groupby('TDC_CHANNEL')['TDC_CHANNEL'].count().nlargest(3)

print('top 3 channels and the corresponding counts: \n ',f)


top 3 channels and the corresponding counts: 
  TDC_CHANNEL
139    44697
64     26928
63     26412
Name: TDC_CHANNEL, dtype: int64


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

In [6]:
a=data.ORBIT_CNT.nunique()
print("Non empyy orbits: \n",a )

Non empyy orbits: 
 5445


7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [7]:
b=data[data['TDC_CHANNEL'].isin([139])].ORBIT_CNT.nunique()
print("number of unique orbits from TDC_CHANNEL=139: \n",b)


number of unique orbits from TDC_CHANNEL=139: 
 5424


8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

In [8]:
#gtupby 0 e 1 e dopo 
data0 = data[data['FPGA'].isin([0])]
s0 = data0.groupby('TDC_CHANNEL')['TDC_CHANNEL'].count()

data1 = data[data['FPGA'].isin([1])]
s1 = data1.groupby('TDC_CHANNEL')['TDC_CHANNEL'].count()
print('FPGA: 0')
print(data0,s0)
print('FPGA: 1')
print(data1,s1)

FPGA: 0
        HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS  \
0          1     0          123  3869200167        2374        26   
1          1     0          124  3869200167        2374        27   
2          1     0           63  3869200167        2553        28   
3          1     0           64  3869200167        2558        19   
4          1     0           64  3869200167        2760        25   
...      ...   ...          ...         ...         ...       ...   
537996     1     0           62  3869205614         749         5   
537997     1     0           64  3869205614         756        21   
537998     1     0          139  3869205614         760         0   
537999     1     0           63  3869205614         758        17   
538000     1     0          120  3869205614         897        25   

         TIME_SERIES  
0       0.000000e+00  
1       8.125000e-01  
2       4.425000e+03  
3       4.550812e+03  
4       9.600812e+03  
...              ...  
53

9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.